#00. Imports

In [101]:
!pip install yahoo_fin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [102]:
from google.colab import drive
from keras.models import load_model
import tensorflow as tf
import pandas as pd
from tensorflow import keras as k
from yahoo_fin import stock_info as si
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [103]:
import pandas as pd

In [104]:
import numpy as np

In [105]:
from sklearn.preprocessing import StandardScaler

#A. Load Models

Contains models of LSTM of variants Boole and Albert

##1. Load Model v11[Albert]

In [106]:
def rmse(y_true, y_pred):
  return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))

In [107]:
model_albert = load_model("/content/drive/MyDrive/Models/Model_V11.h5", custom_objects={'loss': rmse }, compile=False)

In [108]:
model_albert.compile(loss = rmse, optimizer= k.optimizers.Adam(learning_rate=0.01))
early_stopping = k.callbacks.EarlyStopping(monitor='loss', patience=10)
model_albert.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (None, 24, 50)            11200     
                                                                 
 lstm_22 (LSTM)              (None, 24, 50)            20200     
                                                                 
 lstm_23 (LSTM)              (None, 50)                20200     
                                                                 
 dropout_6 (Dropout)         (None, 50)                0         
                                                                 
 dense_6 (Dense)             (None, 24)                1224      
                                                                 
Total params: 52,824
Trainable params: 52,824
Non-trainable params: 0
_________________________________________________________________


In [109]:
str(type(model_albert)) == "<class 'keras.engine.sequential.Sequential'>"

True

##2. Load Model v2[Boole]

In [110]:
model_boole = load_model("/content/drive/MyDrive/Models/Model_V2", custom_objects={'loss': rmse }, compile=False)

In [111]:
model_boole.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 24, 240)           235200    
                                                                 
 lstm_19 (LSTM)              (None, 24, 120)           173280    
                                                                 
 lstm_20 (LSTM)              (None, 60)                43440     
                                                                 
 dropout_6 (Dropout)         (None, 60)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 61        
                                                                 
Total params: 451,981
Trainable params: 451,981
Non-trainable params: 0
_________________________________________________________________


#B. Prepare and load Data

Contains methods to gather and preprocess data

In [112]:
df = si.get_data('NG=F')['close']

In [113]:
df = df.round(3)
df = df.resample('1D').mean()
df.interpolate(method='linear', axis = 0, inplace= True, limit_direction='forward')

In [114]:
df.head(5)

2000-08-30    4.80500
2000-08-31    4.78000
2000-09-01    4.83500
2000-09-02    4.86625
2000-09-03    4.89750
Freq: D, Name: close, dtype: float64

##1. Data Prep v11[Albert]

In [115]:
df_m_albert = pd.DataFrame(df.resample('1M').mean())

In [116]:
type(df_m_albert)

pandas.core.frame.DataFrame

In [117]:
df_m_albert['std'] = df_m_albert['close'].rolling(10).std()

In [118]:
df_m_albert.head(15)

,close,std
2000-08-31,4.792500,NaN
2000-09-30,5.122239,NaN
2000-10-31,5.069688,NaN
2000-11-30,5.800333,NaN
2000-12-31,8.706976,NaN
2001-01-31,7.939685,NaN
2001-02-28,5.678107,NaN
2001-03-31,5.194559,NaN
2001-04-30,5.196172,NaN
2001-05-31,4.217532,1.425124


In [119]:
df_m_albert = df_m_albert[9:]

In [120]:
df_m_albert['std'].isna().sum()

0

In [121]:
str(type(df_m_albert))

"<class 'pandas.core.frame.DataFrame'>"

In [122]:
df_m_albert['close_max'] = df_m_albert['close'] + df_m_albert['std']
df_m_albert['close_mim'] = df_m_albert['close'] - df_m_albert['std']

In [123]:
df_m_albert['gradient'] = np.gradient(df_m_albert['close'])
df_m_albert['d_gradient'] = np.gradient(df_m_albert['gradient'])
df_m_albert.drop(columns=['std'], inplace=True)

In [124]:
df_m_albert.head(5)

,close,close_max,close_mim,gradient,d_gradient
2001-05-31,4.217532,5.642656,2.792409,-0.421032,-0.107306
2001-06-30,3.796500,5.328468,2.264532,-0.528339,-0.004181
2001-07-31,3.160855,4.884609,1.437101,-0.429395,0.045318
2001-08-31,2.937710,4.839914,1.035506,-0.437702,0.135520
2001-09-30,2.285450,4.391155,0.179745,-0.158355,0.348322


In [125]:
scaler_albert = StandardScaler()
scaler_albert_obj = scaler_albert.fit(df_m_albert)
scaler_albert_mean = list(map(np.float32, scaler_albert.mean_))
scaler_albert_var = list(map(np.float32, scaler_albert.scale_))

In [126]:
df_albert = scaler_albert.transform(df_m_albert)

In [127]:
type(df_albert)

numpy.ndarray

##2. Data Prep v2[Boole]

In [128]:
df_d_boole = pd.DataFrame(df.resample('1D').mean())

In [129]:
df_d_boole.interpolate(method='linear', axis = 0, inplace= True, limit_direction='forward')

In [130]:
df_d_boole['ma180'] = df_d_boole['close'].rolling(180).mean()
df_d_boole['ma60'] = df_d_boole['close'].rolling(60).mean()
df_d_boole['ma30'] = df_d_boole['close'].rolling(30).mean()

In [131]:
df_d_boole = df_d_boole[270:]

In [132]:
df_m_boole = df_d_boole.resample('1M').mean()

In [133]:
scaler_boole = StandardScaler()
scaler_boole_obj = scaler_boole.fit(df_m_boole)

In [134]:
df_m_boole = scaler_boole.transform(df_m_boole)

In [135]:
df_boole = df_m_boole

In [136]:
type(df_m_boole)

numpy.ndarray

#C. Batch Generator

Contains methods to transform and batch segment data for models

In [137]:
def batch_gen(df, size_x = 24, size_y = 12):
  X = []
  Y = []
  for i in range(size_x, len(df) - size_y):
    X.append(df[i-size_x:i])
    Y.append([item[0] for item in df[i:i+size_y]])

  X = np.array(X)
  Y = np.array(Y)
  return  X, Y

In [138]:
df_boole_X , df_boole_Y = batch_gen(df_boole , 24, 24)
df_albert_X , df_albert_Y = batch_gen(df_albert , 24, 24)

In [139]:
print(type(df_boole_X) ,type(df_boole_Y))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [140]:
print(type(df_albert_X),type(df_albert_Y))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [141]:
df_boole_X.shape

(208, 24, 4)

In [142]:
df_albert_X.shape

(208, 24, 5)

#D. Boosting

In [143]:
class ModelError(Exception):
  def __init__(self, model, message='Error In Model: '):
    self.model = model
    self.message = message + str(df)+ ". Please pass a <class 'keras.engine.sequential.Sequential'>"
    super().__init__(self.message)
  

class ShapeError(Exception):
  def __init__(self, df, message='Shape Mismatch in DataFrame: '):
    self.df = df
    self.message = message+ str(df) + ". Please Load Tensor of Shape(n, 24,6)"
    super().__init__(self.message)

class DatatypeError(Exception):
  def __init__(self, arg, message='Please Upload a numpy array in arg '):
    self.arg = arg
    self.message = message+ str(arg) + " of Shape(n, 24,6)"
    super().__init__(self.message)

_types = {'model': ["<class 'keras.engine.sequential.Sequential'>"], 
          'data': ["<class 'numpy.ndarray'>", "<class 'pandas.core.frame.DataFrame'>"]
        }

In [170]:
def model_1_fn(model_1, data_model_1, scaler_1):
  x0 = np.reshape(data_model_1[-2] , (1,24,5))
  x = np.reshape(data_model_1[-1] , (1,24,5))
  p=[]
  y_fin=[]
  x_bar = list(map(np.float32, scaler_1.mean_))
  x_std = list(map(np.float32, scaler_1.scale_))
  for index in range(0,3):
    pred_new = model_1.predict(x)
    x0 = [item[0] for item in np.squeeze(x0, axis=0)]
    x = [item[0] for item in np.squeeze(x, axis=0)]
    p = list(np.squeeze(pred_new, axis=0))
    x = x0+x+p
    x = np.array(x)
    x = (x_std[0]* np.array(x)) + x_bar[0]
    y_fin = y_fin + list(x[48:])
    x = pd.DataFrame({'close': x})
    x['std'] = x['close'].rolling(10).std()
    x['close_max'] = x['close'] + x['std']
    x['close_mim'] = x['close'] - x['std']
    x['gradient'] = np.gradient(x['close'])
    x['d_gradient'] = np.gradient(x['gradient'])
    x.drop(columns=['std'], inplace=True)
    x0 = np.reshape(np.array(x[24:48]), (1,24,5))
    x = np.reshape(np.array(x[48:]), (1,24,5))
  return y_fin

72


In [ ]:
def model_2_fn(model_2, data_model_2, scaler_2):
  

In [144]:
def predict(model_1={}, model_2={},data_model_1=np.array([]), data_model_2=np.array([]),scaler_1 = None, scaler_2=None, ratio_albert = 0.50, ratio_boole = 0.50):
  if (str(type(model_1['model'])) not in _types['model']):
    raise ModelError(model_1['name'])

  elif (str(type(model_2['model'])) not in _types['model']):
    raise ModelError(model_2['name'])

  elif (str(type(data_model_1)) not in _types['data']):
    raise DatatypeError('data_model_1')

  elif (str(type(data_model_2)) not in _types['data']):
    raise DatatypeError('data_model_2')

  elif (model_1['input'] == data_model_1.shape[1:]):
    raise ShapeError('data_model_1')

  elif (model_2['input'] == data_model_2.shape[1:]):
    raise ShapeError('data_model_2')

  model_1_pred = model_1_fn(model_1, data_model_1, scaler_1)
  model_2_pred = model_2_fn(model_2, data_model_2, scaler_2)


SyntaxError: ignored

In [146]:
model_albert.predict(np.reshape(df_albert_X[0], (1,24,5)))

array([[0.05400372, 0.06501745, 0.05483598, 0.15447123, 0.19188452,
        0.224985  , 0.27947745, 0.30192184, 0.34102198, 0.35938162,
        0.2980546 , 0.43100786, 0.41511   , 0.4737789 , 0.37954348,
        0.3038459 , 0.2703538 , 0.16170886, 0.13929369, 0.23180136,
        0.33111882, 0.46208397, 0.62037617, 0.740891  ]], dtype=float32)